### We will try to find Boston House price using Lasso, Ridge and ElasticNet Regression Techniques.
### We will also do feature selection using Lasso

In [1]:
# Import Necessary Libraries

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel

In [55]:
# Load the preprocessed data

data = pd.read_csv("boston_EDA.csv")

In [56]:
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,PTRATIO,B,LSTAT,TAX,TARGET
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,15.3,396.90,4.98,296.0,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,17.8,396.90,9.14,242.0,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,17.8,392.83,4.03,242.0,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,18.7,394.63,2.94,222.0,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,18.7,396.90,5.33,222.0,36.2


In [57]:
data.shape

(506, 14)

In all feature selection procedures, it is good practice to select the features by examining only the training set. And this is to avoid overfit.

In [58]:
x_train,x_test,y_train,y_test = train_test_split(data.drop(["TARGET"], axis = 1), data["TARGET"], test_size = 0.3, random_state = 100)

In [59]:
x_train.shape,x_test.shape

((354, 13), (152, 13))

In [60]:
# Do feature Scaling

from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

In [61]:
x_train_scale = scale.fit_transform(x_train)

### Select Features with Lasso

In [78]:
sel = SelectFromModel(Lasso(alpha=1, random_state=10))
sel.fit(x_train_scale,y_train)

SelectFromModel(estimator=Lasso(alpha=1, random_state=10))

In [79]:
sel.get_support()

array([ True, False,  True, False, False,  True, False, False, False,
        True,  True,  True,  True])

In [80]:
sel_features = list(x_train.columns[sel.get_support()])

In [81]:
sel_features

['CRIM', 'INDUS', 'RM', 'PTRATIO', 'B', 'LSTAT', 'TAX']

In [86]:
x_train_data = x_train[sel_features]
x_test_data = x_test[sel_features]

In [88]:
xtrain_scale = scale.fit_transform(x_train_data)
xtest_scale = scale.transform(x_test_data)

### 1. Linear Regression

In [67]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [89]:
lr.fit(xtrain_scale, y_train)

LinearRegression()

In [90]:
y_train_pred = lr.predict(xtrain_scale)

In [91]:
y_test_pred = lr.predict(xtest_scale)

In [92]:
from sklearn.metrics import mean_squared_error

In [93]:
mean_squared_error(y_train,y_train_pred)

19.72433700404926

In [94]:
mean_squared_error(y_test,y_test_pred)

18.975256617390126

### 2. Lasso Regression

In [82]:
from sklearn.linear_model import Lasso, LassoCV

In [104]:
lasscv = LassoCV(alphas = None, cv = 10)
lasscv.fit(xtrain_scale,y_train)

LassoCV(cv=10)

In [105]:
# best alpha parameter
alpha = lasscv.alpha_
alpha

0.03241701642902568

In [106]:
#now that we have best parameter, let's use Lasso regression and see how well our data has fitted before

lasso_reg = Lasso(alpha)
lasso_reg.fit(xtrain_scale, y_train)

Lasso(alpha=0.03241701642902568)

In [107]:
ytrain_pred = lasso_reg.predict(xtrain_scale)

In [108]:
ytest_pred = lasso_reg.predict(xtest_scale)

In [109]:
mean_squared_error(y_train,y_train_pred)

19.72433700404926

In [110]:
mean_squared_error(y_test,y_test_pred)

18.975256617390126

### 3. Ridge Regression

In [101]:
from sklearn.linear_model import Ridge, RidgeCV

In [112]:
alpha = np.random.uniform(low=0, high=10, size=(50,))
ridgecv = RidgeCV(alphas = alpha,cv = 10)
ridgecv.fit(xtrain_scale, y_train)

RidgeCV(alphas=array([4.19019782, 4.25076911, 0.94233228, 0.8486756 , 0.9158549 ,
       1.77285828, 6.83264932, 3.39763994, 7.1079104 , 8.72802624,
       3.07387016, 7.25636541, 5.29391402, 3.44723264, 7.95024773,
       1.33324525, 2.65458854, 6.2270398 , 3.04595462, 6.72714738,
       6.26565422, 0.96498613, 2.04283617, 5.48852534, 8.94337299,
       7.79995388, 8.3533579 , 1.24553642, 1.92211664, 9.42262716,
       4.7400063 , 3.68154099, 0.60229966, 9.02835494, 6.01434362,
       0.06095427, 6.83082852, 1.09360336, 8.95168496, 0.72565077,
       9.88025768, 5.53123649, 6.795853  , 2.4466841 , 1.67322987,
       0.16808926, 6.24127493, 6.221628  , 0.60064669, 7.10065383]),
        cv=10)

In [113]:
ridgecv.alpha_

9.880257679285698

In [114]:
ridge_model = Ridge(alpha = ridgecv.alpha_)
ridge_model.fit(xtrain_scale, y_train)

Ridge(alpha=9.880257679285698)

In [115]:
ytrain_pred = ridge_model.predict(xtrain_scale)

In [116]:
ytest_pred = ridge_model.predict(xtest_scale)

In [117]:
mean_squared_error(y_train,y_train_pred)

19.72433700404926

In [120]:
mean_squared_error(y_test,y_test_pred)

18.975256617390126

### 4. ElasticNet Regression

In [121]:
from sklearn.linear_model import ElasticNet, ElasticNetCV

In [122]:
elcv = ElasticNetCV(cv = 10)
elcv.fit(xtrain_scale, y_train)

ElasticNetCV(cv=10)

In [123]:
elcv.alpha_

0.1302666785077834

In [124]:
elcv.l1_ratio

0.5

In [125]:
el = ElasticNet(alpha = elcv.alpha_,l1_ratio = elcv.l1_ratio)
el.fit(xtrain_scale, y_train)

ElasticNet(alpha=0.1302666785077834)

In [130]:
ytrain_pred = el.predict(xtrain_scale)

In [131]:
ytest_pred = el.predict(xtest_scale)

In [132]:
mean_squared_error(y_train,y_train_pred)

19.72433700404926

In [133]:
mean_squared_error(y_test,y_test_pred)

18.975256617390126